In [50]:
# Import dependencies and setup
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt

In [51]:
# Reflect Tables into SQLAlchemy ORM

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [52]:
# Set engine
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [53]:
# Reflect database
Base = automap_base()

# Reflect tables
Base.prepare(engine, reflect=True)

In [54]:
# View reflected table names
Base.classes.keys()

['measurement', 'station']

In [55]:
# Save each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [56]:
# Create session 
session = Session(engine)

In [59]:
# Exploratory Climate Analysis

# Query for dates and find latest
latestDate = (session.query(Measurement.date)
                     .order_by(Measurement.date.desc())
                     .first())

# Extract string from query object
latestDate = list(np.ravel(latestDate))[0]

# Convert date string to datetime object
latestDate = dt.datetime.strptime(latestDate, '%Y-%m-%d')

#extract year, month, and day as integers
latestYear = int(dt.datetime.strftime(latestDate, '%Y'))
latestMonth = int(dt.datetime.strftime(latestDate, '%m'))
latestDay = int(dt.datetime.strftime(latestDate, '%d'))

# Calculate one year before latest date
yearBefore = dt.date(latestYear, latestMonth, latestDay) - dt.timedelta(days=365)

# Query for dates and precipitation for the latest year
rainData = (session.query(Measurement.date, Measurement.prcp)
                  .filter(Measurement.date > yearBefore)
                  .order_by(Measurement.date)
                  .all())

# Convert query object to data frame and set index
rainTable = pd.DataFrame(rainData)
rainTable = rainTable.set_index('date')

# Sort by date
rainTable = rainTable.sort_index(ascending=True)

# Plot data with pandas
rainTable.plot(title="Precipitation: 8/23/16 - 8/23/17")

KeyError: "None of ['date'] are in the columns"